<a href="https://colab.research.google.com/github/yingzibu/MOL2AE/blob/main/examples/FP_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### NEED TO DO

1. train:val:test split. Not done here. Used all drugs to do training (since the data is few)
2. label should be one hot encoding and use BCELoss, here just use the original [1,2,3,4,5] and used MSELoss, need to be changed
3. Did not incorporate protein expression info. Could try simple multiplication on IC$_{50}$


In [ ]:
cd /content/drive/MyDrive/tox_pred

/content/drive/MyDrive/tox_pred


In [ ]:
ls

drug_reaction_freq.tsv  drug_smiles_ic50.tsv  hpa_gene_seqs.json


In [ ]:
! rm -R 'JAK_ML'
! git clone https://github.com/yingzibu/JAK_ML.git --quiet
!rm -R 'a_inhibitor_design'
! git clone https://github.com/yingzibu/a_inhibitor_design.git --quiet
! pip install pubchempy --quiet
! pip install rdkit --quiet

In [ ]:
import pandas as pd
from tqdm import tqdm
drug_freq = pd.read_table('drug_reaction_freq.tsv')
drug_freq = drug_freq.iloc[:, :-1]
drug_smiles = pd.read_table('drug_smiles_ic50.tsv')
drug_smiles = drug_smiles.iloc[:, :-1]

In [ ]:
df_all = drug_smiles.merge(drug_freq)

In [ ]:
# JUST SELETED A SUBSET OF IC50 AND FREQUENCY TO TEST CODE
small_drug_smiles = drug_smiles.iloc[:, :10]
small_drug_freq = drug_freq.iloc[:, :10]
small_df = small_drug_smiles.merge(small_drug_freq)

small_df

In [ ]:
from JAK_ML.ML_code.function import *
def process(df):
    MACCS_list = smile_list_to_MACCS(df['SMILES'].tolist())
    header = ['bit' + str(i) for i in range(167)]
    new_df = pd.DataFrame(MACCS_list, columns=header)
    new_df['SMILES'] = df['SMILES']
    new_df = new_df.merge(df)
    return new_df
data_df = process(df_all)

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import pickle
import numpy as np

class Classifier(nn.Module):
    def __init__(self, dims):
        super(Classifier, self).__init__()
        [in_dim, h_dims, out_dim] = dims
        self.dims = dims
        neurons = [in_dim, *h_dims]
        linear_layers = [nn.Linear(neurons[i-1], neurons[i]) \
                         for i in range(1, len(neurons))]
        self.hidden = nn.ModuleList(linear_layers)
        # self.emb = nn.GRU(h_dims[-1], h_dims[-1])
        self.final = nn.Linear(h_dims[-1], out_dim)

    def forward(self, x):
        for layer in self.hidden:
            x = F.relu(layer(x))
        x = self.final(x)
        return x

def onehot(k):
    def encode(label):
        y = torch.zeros(len(label), k+1)
        for i, j  in enumerate(label):
            if j.item() <= k: y[i][int(j.item())] = 1
        return y[:, 1:]
    return encode
# onehot(5)(ae)

class tox_dataset(Dataset):
    def __init__(self, df):
        self.len = len(df)
        self.df = df
        self.ic_start_ind = df.columns.get_loc("TSPAN6")
        self.ae_start_ind = df.columns.get_loc('asthenia')

    def __getitem__(self, idx):
        """
        OUTPUT
        :param fp: fingerprint, should be 167 dim
        :param ic: ic50
        :param ae: adverse events
        """
        header = ['bit' + str(i) for i in range(167)]
        fp = self.df[header]
        fp = torch.tensor([float(b) for b in fp.iloc[idx]], dtype=torch.float32)
        ic = self.df.iloc[:, self.ic_start_ind:self.ae_start_ind]
        ic = torch.tensor(ic.values.astype(np.float32))[idx]
        ae = self.df.iloc[:, self.ae_start_ind:]
        ae = torch.tensor(ae.values.astype(np.float32))[idx]
        # ae = onehot(5)(ae)
        return fp, ic, ae
    def __len__(self): return self.len


In [ ]:
# dataset test, select first row
fp, ic, ae = tox_dataset(data_df)[0]
fp, ic, ae

In [ ]:
# hidden dims of neural network are changeable,
# as long as its all integers
h_dims = [256*2, 256, 128]
##############################################


# BELOW IS NOT CHANGEABLE
in_dim = fp.shape[0] + ic.shape[0]
out_dim = ae.shape[0]

dims = [in_dim, h_dims, out_dim]
dims

[15171, [512, 256, 128], 994]

In [ ]:
# batch_size is changeable
params = {'batch_size':16, 'shuffle':True,
          'drop_last':False, 'num_workers': 0}

train_loader = DataLoader(tox_dataset(data_df), **params)

In [ ]:
model = Classifier(dims)
cuda = torch.cuda.is_available()
if cuda: model = model.cuda()

In [ ]:
import torch.optim as optim
loss_function = nn.MSELoss() # better use cross entropy, need onehot encoding

# parameter changeable:
lr = 1e-3 # learning rate
wd = 1e-3 # weight decay
#######################

optimizer = optim.AdamW(params=model.parameters(), lr=lr, weight_decay=wd)

In [ ]:
import matplotlib.pyplot as plt
def get_min(d:dict):
    min_key = next(iter(d))
    for key in d:
        if d[key] < d[min_key]: min_key = key
    return min_key, d[min_key]

def plot_loss(a:dict):
    plt.plot(a.keys(), a.values())
    argmin, min = get_min(a)
    plt.plot(argmin, min, '*', label=f'min epoch: {argmin}')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss during training')
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
epochs = 1000
min_loss = 100000

loss_dict = {}
for epoch in range(epochs):
    total_loss = 0
    model.train()
    for i, (fp, ic, ae) in tqdm(enumerate(train_loader),
                                total=len(train_loader),
                            desc=f'Epoch {epoch}'):
        optimizer.zero_grad()
        if cuda: fp, ic, ae = fp.cuda(), ic.cuda(), ae.cuda()
        mask = ae == -100
        output = model(torch.cat((fp, ic), 1))
        loss = loss_function(output[~mask], ae[~mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    total_loss /= len(train_loader)
    print(f'[Train] MSE loss: {total_loss:.3f}')
    loss_dict[epoch] = total_loss
    if total_loss < min_loss:
        min_loss = total_loss
        torch.save(model.state_dict(), 'ckpt.pt')

    # Employ early stop
    # training too long yet no loss decrease:  STOP
    argmin, min = get_min(loss_dict)
    if epoch - argmin > 10:
        print('no decrease after 10 epoch, early stop')
        break

    # training yet see loss increase too much: STOP
    if total_loss / min > 1.3:
        print('loss increase 30% compared with min loss, early stop')
        break

    if epoch % 10 == 0 and epoch != 0:
        plot_loss(loss_dict)
